In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)

W0713 17:32:26.520805 24928 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0713 17:32:26.550759 24928 deprecation_wrapper.py:119] From e:\anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(4.7784)
tensor(8.9947)
tensor(5.1467)
tensor(1.0028)
tensor(0.8636)
tensor(0.7977)
tensor(0.7314)
tensor(0.6604)
tensor(0.5870)
tensor(0.5057)
tensor(0.4294)
tensor(0.3494)
tensor(0.2806)
tensor(0.2234)
tensor(0.1853)
tensor(0.1332)
tensor(0.1098)
tensor(0.0783)
tensor(0.0597)
tensor(0.0480)


In [3]:
model(data)

tensor([[0.0286],
        [0.0211],
        [0.9675],
        [0.8313]], grad_fn=<AddmmBackward>)

In [4]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [5]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [7]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:69693572539 -> alice:22409246242]
 	-> (Wrapper)>[PointerTensor | me:60350907333 -> bob:20289854630]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:36650920271 -> alice:55582624183]
 	-> (Wrapper)>[PointerTensor | me:14158351457 -> bob:59314719522]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:21843988824 -> alice:31209940001]
 	-> (Wrapper)>[PointerTensor | me:5751722988 -> bob:7123246156]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:77394830821 -> alice:44181157196]
 	-> (Wrapper)>[PointerTensor | me:52022807119 -> bob:1243970954

In [8]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [9]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23386493690 -> alice:52123971837]
	-> (Wrapper)>[PointerTensor | me:80299489696 -> bob:81440798170]
	*crypto provider: secure_worker*

In [10]:
encrypted_prediction = encrypted_model(encrypted_data)

In [11]:
encrypted_prediction.get().float_precision()

tensor([[0.0310],
        [0.0230],
        [0.9680],
        [0.8310]])